In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import csv
from numpy import genfromtxt
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

print("GPU: ", tf.test.is_gpu_available())

# data preprocessing and data path
data = genfromtxt("/content/drive/My Drive/訓練資料/America/LS.csv",  delimiter=",", dtype=str)
#print(data)
#print(data.shape)

data = data.astype(np.float32)
data2 = data[:,1]

data2 = np.reshape(data2, (data2.shape[0], 1))
print(data2)
scaler = MinMaxScaler(feature_range=(0,1))
data2 = scaler.fit_transform(data2)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:i+look_back,0]
        dataX.append(a)
        dataY.append(dataset[i+look_back,0])
    return np.array(dataX), np.array(dataY)


train_size = int(len(data2) * 0.67)
look_back = 1
train_dataX, train_dataY = create_dataset(data2[0:train_size,:], look_back)
test_dataX, test_dataY = create_dataset(data2[train_size:len(data2)], look_back)


train_dataX = np.reshape(train_dataX, (train_dataX.shape[0], 1, train_dataX.shape[1]))
test_dataX = np.reshape(test_dataX, (test_dataX.shape[0], 1, test_dataX.shape[1]))
print("data2.shape: ", data2.shape)
print("train_dataY: ", train_dataY.shape)


model = Sequential()
model.add(LSTM(4, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mse', optimizer="adam")

#訓練模型參數設定

history = model.fit(train_dataX, train_dataY, epochs=5, batch_size=1, verbose=1)

model.save('lstm.h5')
trainPredict = model.predict(train_dataX)
testPredict = model.predict(test_dataX)


trainScore = math.sqrt(mean_squared_error(train_dataY, trainPredict[:,0]))
testScore = math.sqrt(mean_squared_error(test_dataY, testPredict[:,0]))
print('TrainScore: %.2f RMSE(norm)' % (trainScore))
print('TestScore: %.2f RMSE(norm)' % (testScore))

# 恢復原始數據 scale
trainPredict = scaler.inverse_transform(trainPredict)
train_dataY = scaler.inverse_transform([train_dataY])
testPredict = scaler.inverse_transform(testPredict)
test_dataY = scaler.inverse_transform([test_dataY])
#print('testPredict = ', (testPredict))
#print('testPredict = ', (test_dataY))
print('dataTpye：',type(testPredict))
print('dataTpye：',type(test_dataY))

df0 = pd.DataFrame(testPredict,columns=['predict']) #ndarray 轉dataframe
df1 = pd.DataFrame(test_dataY.T)

df0['test_Y'] = df1

dataname="X.csv"  #savefilename
df0.to_csv(dataname,index=0)


trainPredictPlot = np.empty_like(data2)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# 畫測試資料趨勢圖
# shift test predictions for plotting
testPredictPlot = np.empty_like(data2)
testPredictPlot[:, :] = np.nan 
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data2)-1, :] = testPredict

# 畫原始資料趨勢圖
#plot baseline and predictions
plt.plot(scaler.inverse_transform(data2))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()